In [14]:
import pandas as pd 
import numpy as np

## 1. Carga fichero.

In [15]:
df = pd.read_csv("../Data/train.csv")
df.head()

,work_year,salario_eur,experience_level_mean,employment_type_mean,job_title_mean,company_location_mean,employee_location_mean,company_size_mean,remote_ratio_mean
0,2022,33831.16,140806.985819,127249.063604,131362.875846,53061.979740,55626.282000,131680.104497,127249.063604
1,2023,139656.00,96163.864000,127249.063604,148317.059512,139676.248776,140596.250719,131680.104497,127249.063604
2,2022,92000.00,140806.985819,127249.063604,100018.928954,139676.248776,140596.250719,131680.104497,127249.063604
3,2022,144072.00,140806.985819,127249.063604,131362.875846,139676.248776,140596.250719,131680.104497,127249.063604
4,2023,161000.00,140806.985819,127249.063604,131362.875846,139676.248776,140596.250719,131680.104497,127249.063604


In [16]:
df.describe()

,work_year,salario_eur,experience_level_mean,employment_type_mean,job_title_mean,company_location_mean,employee_location_mean,company_size_mean,remote_ratio_mean
count,3004.000000,3004.000000,3004.000000,3004.000000,3004.000000,3004.000000,3004.000000,3004.000000,3004.000000
mean,2022.367177,127011.510200,126571.201710,126563.919420,126934.830434,126748.237500,126770.925641,126630.927094,126563.919420
std,0.691994,58054.043449,25848.609445,7385.145193,21868.171718,30119.169160,31244.172219,13169.938111,7385.145193
min,2020.000000,4721.440000,72262.581625,36371.009412,19644.070000,5799.680000,5799.680000,71968.547838,36371.009412
25%,2022.000000,87400.000000,96163.864000,127249.063604,129600.184238,139676.248776,140596.250719,131680.104497,127249.063604
50%,2022.000000,124200.000000,140806.985819,127249.063604,131362.875846,139676.248776,140596.250719,131680.104497,127249.063604
75%,2023.000000,162150.000000,140806.985819,127249.063604,131362.875846,139676.248776,140596.250719,131680.104497,127249.063604
max,2023.000000,414000.000000,179336.455439,127249.063604,230000.000000,249730.780000,389927.280000,131680.104497,127249.063604


## 2. Creacion X/Y.

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X = df.drop(columns=["salario_eur"], axis=1)
y = df["salario_eur"]

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(600, 8)
(2404, 8)
(600,)
(2404,)


### 3. Modelo baseline.

In [20]:
from sklearn.linear_model import LinearRegression

In [61]:
baseline_model = LinearRegression()
baseline_model.fit(X_train, y_train)

LinearRegression()

In [62]:
baseline_predictions = baseline_model.predict(X_test)

In [39]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score

In [63]:
print("MAE", mean_absolute_error(y_test, baseline_predictions))
print("MSE", mean_squared_error(y_test, baseline_predictions))
print("RMSE", np.sqrt(mean_squared_error(y_test, baseline_predictions)))

MAE 33441.58526043329
MSE 2027566593.1896944
RMSE 45028.50867161486


#### 4. Polynomial Regression

In [52]:
from sklearn.preprocessing import PolynomialFeatures

poly_reg = PolynomialFeatures(degree=3)
poly_reg.fit(X_train)
X_train_poly = poly_reg.transform(X_train)
X_test_poly = poly_reg.transform(X_test)

pol_reg_2 = LinearRegression()
pol_reg_2.fit(X_train_poly, y_train)

LinearRegression()

In [53]:
poly_prediction = pol_reg_2.predict(X_test_poly)

print("scoore" , pol_reg_2.score(X_train_poly, y_train))

print("MAE", mean_absolute_error(y_test, poly_prediction))
print("MSE", mean_squared_error(y_test, poly_prediction))
print("RMSE", np.sqrt(mean_squared_error(y_test, poly_prediction)))


scoore 0.5555791898677631
MAE 375123155.5069547
MSE 3.739585931336453e+19
RMSE 6115215393.8650875


Mejora el resultado pero sigue dando margen a mejora.

#### 4. Regression Tree

In [33]:
from sklearn.tree import DecisionTreeRegressor

In [34]:
from sklearn.model_selection import GridSearchCV

model = DecisionTreeRegressor(random_state=0)

parameters = {"max_depth":[2,3,4,5,6,7],
              "criterion": ['squared_error', 'absolute_error'],
              "min_samples_split": [2,4,6,8,10,12],
              "min_samples_leaf": [2,5,10,15],
              "max_features": [1,2]
}

dtr_gs = GridSearchCV(model, parameters, cv=5, scoring="neg_mean_absolute_error", n_jobs= -1)

In [35]:
dtr_gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=0), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'absolute_error'],
                         'max_depth': [2, 3, 4, 5, 6, 7],
                         'max_features': [1, 2],
                         'min_samples_leaf': [2, 5, 10, 15],
                         'min_samples_split': [2, 4, 6, 8, 10, 12]},
             scoring='neg_mean_absolute_error')

In [36]:
dtr_gs.best_params_

{'criterion': 'absolute_error',
 'max_depth': 7,
 'max_features': 2,
 'min_samples_leaf': 2,
 'min_samples_split': 6}

In [48]:
y_pred_tree = dtr_gs.best_estimator_.predict(X_test)
print("scoore" , pol_reg_2.score(X_train_poly, y_train))
print("MAE", mean_absolute_error(y_test, y_pred_tree))
print("MSE", mean_squared_error(y_test, y_pred_tree))
print("RMSE", np.sqrt(mean_squared_error(y_test, y_pred_tree)))

scoore 0.5555791898677631
MAE 34541.95586522463
MSE 2301249167.869171
RMSE 47971.33694060622


In [9]:
import pickle

with open('../models/train_model.pkl', 'wb') as archivo_salida:
    pickle.dump(baseline_model, archivo_salida)

### XGRB modelo


In [42]:

import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.preprocessing import OrdinalEncoder


In [45]:
model_xgb = XGBRegressor(
    objective="reg:squarederror",
    n_estimators=10_000,
    max_depth=7,
    learning_rate=0.1,
)

In [46]:
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=7, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=10000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [71]:
y_pred_xgb = model_xgb.predict(X_test)
print("scoore" , model_xgb.score(X_test, y_test))
print("MAE", mean_absolute_error(y_test, y_pred_xgb))
print("MSE", mean_squared_error(y_test, y_pred_xgb))
print("RMSE", np.sqrt(mean_squared_error(y_test, y_pred_xgb)))

scoore 0.34310271050247376
MAE 34750.11931757618
MSE 2245766882.579237
RMSE 47389.52291993703


### RANDOM FOREST REGRESION

In [87]:
from sklearn.ensemble import RandomForestRegressor
RFR_model = RandomForestRegressor(max_depth=5, n_estimators=10)



In [88]:
RFR_model.fit(X_train, y_train)

RandomForestRegressor(max_depth=5, n_estimators=10)

In [85]:
from sklearn.metrics import r2_score

In [94]:
y_pred_RFR= RFR_model.predict(X_test)
print("scoore train" , RFR_model.score(X_train, y_train))
print("scoore test" , RFR_model.score(X_test, y_test))

print("MAE", mean_absolute_error(y_test, y_pred_RFR))
print("MSE", mean_squared_error(y_test, y_pred_RFR))
print("RMSE", np.sqrt(mean_squared_error(y_test, y_pred_RFR)))
print("RF_scoore", r2_score(y_test , y_pred_RFR))

scoore train 0.5101080212731279
scoore test 0.4625277931254358
MAE 31921.26206427934
MSE 1837482513.3301423
RMSE 42865.86652956105
RF_scoore 0.4625277931254358


## PIPELINE


In [79]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [80]:

# Definir los pasos del pipeline
steps = [
    ('scaler', StandardScaler()),  # Escalado de datos
    ('model', None)  # Modelo a evaluar (se especificará posteriormente)
]

# Crear el pipeline
pipeline = Pipeline(steps)

# Definir los hiperparámetros a ajustar para cada modelo
parameters = [
    {
        'model': [LinearRegression()]  # Regresión lineal
    },
    {
        'model': [RandomForestRegressor()],  # RandomForestRegressor
        'model__n_estimators': [10, 50, 100],  # Número de árboles en el bosque
        'model__max_depth': [None, 5, 10],  # Profundidad máxima de los árboles
    },
    {
        'model': [XGBRegressor()],  # XGBRegressor
        'model__n_estimators': [10, 50, 100],  # Número de estimadores (árboles)
        'model__max_depth': [3, 6, 9],  # Profundidad máxima de los árboles
        'model__learning_rate': [0.1, 0.01, 0.001],  # Tasa de aprendizaje
    }
]

# Realizar la búsqueda de hiperparámetros y seleccionar el mejor modelo
grid_search = GridSearchCV(pipeline, parameters, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)  # X_train y y_train son tus datos de entrenamiento

# Obtener el mejor modelo y sus hiperparámetros
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Imprimir el mejor modelo y sus hiperparámetros
print("Mejor modelo:", best_model)
print("Mejores hiperparámetros:", best_params)

Mejor modelo: Pipeline(steps=[('scaler', StandardScaler()),
                ('model', RandomForestRegressor(max_depth=5, n_estimators=10))])
Mejores hiperparámetros: {'model': RandomForestRegressor(max_depth=5, n_estimators=10), 'model__max_depth': 5, 'model__n_estimators': 10}
